# Image_Classifier

In [15]:
from PIL import Image
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import csv
%matplotlib inline

In [16]:
def warn(*args,**kwargs): pass
import warnings
warnings.warn=warn

In [23]:
from sklearn.preprocessing import LabelEncoder
# help('sklearn')
from sklearn.model_selection import StratifiedShuffleSplit

## Parsing the images into CSV

In [12]:
# #Useful function
# def createFileList(myDir, format='.jpg'):
#     fileList = []
#     print(myDir)
#     for root, dirs, files in os.walk(myDir, topdown=False):
#         for name in files:
#             if name.endswith(format):
#                 fullName = os.path.join(root, name)
#                 fileList.append(fullName)
#     return fileList

# # load the original image
# myFileList = createFileList('/home/sambasivarao/Pictures/headshot')

# for file in myFileList:
#     print(file)
#     img_file = Image.open(file)
#     # img_file.show()

#     # get original image parameters...
#     width, height = img_file.size
#     format = img_file.format
#     mode = img_file.mode

#     # Make image Greyscale
#     img_grey = img_file.convert('L')
#     #img_grey.save('result.png')
#     #img_grey.show()

#     # Save Greyscale values
#     value = np.asarray(img_grey.getdata(), dtype=np.int).reshape((img_grey.size[1], img_grey.size[0]))
#     value = value.flatten()
#     print(value)
#     with open("img_pixels.csv", 'a') as f:
#         writer = csv.writer(f)
#         writer.writerow(value)

## Reading in the training data and test data

In [42]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Columns: 194 entries, id to texture64
dtypes: float64(192), int64(1), object(1)
memory usage: 1.5+ MB


In [26]:
train.head()

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [30]:
train.species.value_counts()

Populus_Grandidentata           10
Cornus_Controversa              10
Viburnum_x_Rhytidophylloides    10
Quercus_Ellipsoidalis           10
Acer_Rufinerve                  10
Quercus_Castaneifolia           10
Quercus_Nigra                   10
Acer_Saccharinum                10
Ulmus_Bergmanniana              10
Crataegus_Monogyna              10
Quercus_Variabilis              10
Fagus_Sylvatica                 10
Quercus_Rhysophylla             10
Acer_Pictum                     10
Quercus_Imbricaria              10
Salix_Intergra                  10
Liriodendron_Tulipifera         10
Quercus_Dolicholepis            10
Liquidambar_Styraciflua         10
Alnus_Cordata                   10
Prunus_X_Shmittii               10
Cercis_Siliquastrum             10
Quercus_Pyrenaica               10
Castanea_Sativa                 10
Olea_Europaea                   10
Quercus_Crassipes               10
Alnus_Viridis                   10
Quercus_Vulcanica               10
Quercus_x_Hispanica 

In [29]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Columns: 193 entries, id to texture64
dtypes: float64(192), int64(1)
memory usage: 895.7 KB


In [28]:
test.head()

,id,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,4,0.019531,0.009766,0.078125,0.011719,0.003906,0.015625,0.005859,0.0,0.005859,...,0.006836,0.000000,0.015625,0.000977,0.015625,0.0,0.0,0.000000,0.003906,0.053711
1,7,0.007812,0.005859,0.064453,0.009766,0.003906,0.013672,0.007812,0.0,0.033203,...,0.000000,0.000000,0.006836,0.001953,0.013672,0.0,0.0,0.000977,0.037109,0.044922
2,9,0.000000,0.000000,0.001953,0.021484,0.041016,0.000000,0.023438,0.0,0.011719,...,0.128910,0.000000,0.000977,0.000000,0.000000,0.0,0.0,0.015625,0.000000,0.000000
3,12,0.000000,0.000000,0.009766,0.011719,0.017578,0.000000,0.003906,0.0,0.003906,...,0.012695,0.015625,0.002930,0.036133,0.013672,0.0,0.0,0.089844,0.000000,0.008789
4,13,0.001953,0.000000,0.015625,0.009766,0.039062,0.000000,0.009766,0.0,0.005859,...,0.000000,0.042969,0.016602,0.010742,0.041016,0.0,0.0,0.007812,0.009766,0.007812


## Encoding all the species

In [38]:
le = LabelEncoder().fit(train.species)
labels = le.transform(train.species)
labels


0         4
1         7
2         9
3        12
4        13
5        16
6        19
7        23
8        24
9        28
10       33
11       36
12       39
13       41
14       44
15       46
16       47
17       51
18       52
19       53
20       57
21       59
22       62
23       65
24       68
25       70
26       74
27       77
28       79
29       86
       ... 
564    1493
565    1495
566    1497
567    1498
568    1503
569    1510
570    1513
571    1517
572    1522
573    1526
574    1528
575    1533
576    1534
577    1535
578    1537
579    1540
580    1542
581    1546
582    1553
583    1558
584    1560
585    1564
586    1565
587    1567
588    1573
589    1576
590    1577
591    1579
592    1580
593    1583
Name: id, Length: 594, dtype: int64

In [39]:
def encode(train,test):
    le = LabelEncoder().fit(train.species)
    labels = le.transform(train.species)
    classes = le.classes_
    test_ids = test.id
    
    train = train.drop(['species','id'],axis=1)
    test = test.drop(['id'],axis=1)
    
    return train, labels, test, test_ids, classes
    

In [43]:
train, labels, test, test_ids, classes = encode(train,test)
train.head()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,0.033203,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,0.007812,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,0.044922,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,0.017578,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,0.005859,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


## Stratified Sampling